# Day1-02: Agent 架构模式

学习三种多 Agent 协作模式：动态协调、顺序执行、并行处理。

## 步骤 1: 配置 API Key

In [96]:
import os
from pathlib import Path

# 读取项目根目录的 .env 文件
env_file = Path.cwd().parent / '.env'

if env_file.exists():
    for line in env_file.read_text().splitlines():
        if line.startswith('GOOGLE_API_KEY='):
            os.environ["GOOGLE_API_KEY"] = line.split('=', 1)[1].strip()
            print("✅ Gemini API key setup complete.")
            break
else:
    print(f"⚠️ Please create .env file at: {env_file}")


✅ Gemini API key setup complete.


## 步骤 2: 导入组件
导入多 Agent 架构所需组件：`SequentialAgent`（顺序）、`ParallelAgent`（并行）

In [97]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


## 步骤 3: 配置重试策略
设置 HTTP 请求的重试选项，确保 API 调用稳定性

In [98]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

## 步骤 4: 创建 Agent
定义一个具备 Google Search 能力的助手 Agent

In [99]:
root_agent = Agent(
    name="helpful_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


## 步骤 5: 创建 Runner
Runner 负责执行 Agent 的推理和工具调用流程

In [100]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

✅ Runner created.


## 多 Agent 系统 vs 单 Agent

- **单 Agent**：瑞士军刀，样样通，样样松
- **多 Agent**：专业团队，分工协作，高效精准

## 示例 1: 动态协调 - 研究总结系统

两个专业 Agent：研究员（搜索）→ 总结员（提炼）

In [101]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
    You are a specialized research assistant. Your task is to use the Google search tool to gather 2-3 relevant, credible sources of information on the given topic. Once you’ve found the information, present the key findings along with proper citations for each source.
    """,
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

✅ research_agent created.


In [102]:
# Summarizer Agent: Its job is to summarize the text it receives.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""
    Review the provided research findings: {research_findings}.
    Summarize the key points in a concise bulleted list, highlighting 3-5 of the most important insights.
    """,
    output_key="final_summary",
)

print("✅ summarizer_agent created.")

✅ summarizer_agent created.


### 创建协调者 Agent
根 Agent 动态调用子 Agent，使用 `AgentTool` 封装

In [103]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""
    You are a research coordinator responsible for efficiently answering the user's query through a structured workflow:
    Initiate Research: Begin by using the `ResearchAgent` tool to gather relevant information on the topic provided by the user.
    Summarize Findings: After receiving the research results, use the `SummarizerAgent` tool to generate a clear and concise summary of the key points.
    Present Final Response: Share the final summary with the user, ensuring the information is presented in a straightforward and easily understandable manner.
    """,
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("✅ root_agent created.")

✅ root_agent created.


### 测试系统

In [104]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "人形机器人最新进展有哪些？它们对人类意味着什么？"
)


 ### Created new session: debug_session_id

User > 人形机器人最新进展有哪些？它们对人类意味着什么？


**关键点**：协调者 Agent 根据任务动态决策，调用合适的子 Agent

## 示例 2: 顺序执行 - 博客创作流水线

三个 Agent 顺序协作：大纲 → 写作 → 编辑

In [105]:
# 大纲Agent： 负责创建初始博客文章大纲。
outline_agent = Agent(
    name="OutlineAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline",  # The result of this agent will be stored in the session state with this key.
)

print("✅ outline_agent created.")

✅ outline_agent created.


In [106]:
# 写作Agent： 负责根据上一个Agent提供的大纲撰写完整的博客文章。
writer_agent = Agent(
    name="WriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The `{blog_outline}` placeholder automatically injects the state value from the previous agent's output.
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 800 to 1000-word blog post with an engaging and informative tone.""",
    output_key="blog_draft",  # The result of this agent will be stored with this key.
)

print("✅ writer_agent created.")

✅ writer_agent created.


In [107]:
# 编辑Agent： 负责对作者代理的草稿进行编辑和润色。
editor_agent = Agent(
    name="EditorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This agent receives the `{blog_draft}` from the writer agent's output.
    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog",  # This is the final output of the entire pipeline.
)

print("✅ editor_agent created.")

✅ editor_agent created.


### 创建顺序 Agent
使用 `SequentialAgent` 按顺序执行

In [108]:
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


### 测试流水线

In [109]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "写关于一篇对传统产品经理转型 AI 产品经理的好处的博客文章"
)


 ### Created new session: debug_session_id

User > 写关于一篇对传统产品经理转型 AI 产品经理的好处的博客文章
OutlineAgent > 好的，这是一个关于传统产品经理转型AI产品经理好处的博客文章大纲：

## 博客文章大纲：传统产品经理转型 AI 产品经理的好处

**1. 标题：**

**从像素到智能：传统产品经理拥抱 AI 的华丽蜕变之旅**

**2. 引言钩子：**

你是否曾坐在会议室里，看着产品指标的起伏，思考着下一个“杀手级”功能？如果是，那么你可能正在经历一场无声的革命。随着人工智能（AI）以前所未有的速度渗透到各个行业，传统的产品经理们正面临一个关键的十字路口。拥抱AI，不仅仅是学习一项新技术，更是一次对职业生涯的深刻重塑，带来前所未有的机遇和价值。

**3. 主要部分：**

*   **第一部分：解锁前所未有的产品创新潜力**
    *   **AI驱动的洞察力：** 传统产品经理依赖用户访谈和数据分析，而AI能挖掘更深层次、更复杂的用户行为模式，预测趋势，甚至发现未被满足的需求。
    *   **智能化用户体验：** 想象一下，产品能根据用户习惯主动调整界面、推荐内容，甚至预测用户意图。AI产品经理能创造真正个性化、自适应的极致用户体验。
    *   **自动化与效率提升：** AI不仅能优化产品功能，还能自动化许多产品管理流程，如A/B测试的优化、用户反馈的初步分析等，让产品经理聚焦更具战略性的工作。

*   **第二部分：提升职业竞争力与市场价值**
    *   **拥抱未来趋势，站立行业前沿：** AI是未来的核心驱动力。转型AI产品经理，意味着你不再是追随者，而是站在技术变革的最前线，具备引领行业发展的能力。
    *   **更高薪资与更广阔的职业前景：** 市场对具备AI产品管理技能的人才需求旺盛，这直接转化为更高的薪资水平和更多元的职业发展路径，从初创公司到大型科技巨头，机会无限。
    *   **解决更复杂、更有影响力的挑战：** AI产品经理有机会解决医疗、环保、教育等领域的重大社会问题，创造出真正具有深远影响力的解决方案，实现个人价值和社会价值的双重提升。

*   **第三部分：重塑产品思维与方法论**
    *   **数据驱动到

## 示例 3: 并行处理 - 多领域研究

三个研究 Agent 并行工作 + 一个汇总 Agent：科技 + 健康 + 金融 → 综合报告

In [110]:
# Agent1：科技研究员，专注于人工智能和机器学习趋势
tech_researcher = Agent(
    name="TechResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research",  # The result of this agent will be stored in the session state with this key.
)

print("✅ tech_researcher created.")

✅ tech_researcher created.


In [111]:
# Agent2：健康研究员，专注于医学突破。
health_researcher = Agent(
    name="HealthResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research",  # The result will be stored with this key.
)

print("✅ health_researcher created.")

✅ health_researcher created.


In [112]:
# Agent3：金融研究员，专注于金融科技趋势。
finance_researcher = Agent(
    name="FinanceResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research",  # The result will be stored with this key.
)

print("✅ finance_researcher created.")

✅ finance_researcher created.


In [113]:
# Agent4：聚合器代理在并行步骤之后运行，以综合结果。
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


### 创建混合架构
并行研究 → 顺序汇总（嵌套结构）

In [114]:
# ParallelAgent 同时运行其所有子代理。
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

# 这个SequentialAgent定义了高级工作流程：首先运行并行团队，然后运行聚合器。
root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

✅ Parallel and Sequential Agents created.


### 测试并行系统

In [115]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "今日科技、健康和金融高管简报，使用中文输出"
)


 ### Created new session: debug_session_id

User > 今日科技、健康和金融高管简报，使用中文输出
TechResearcher > **最新AI/ML趋势简报**

**1. Agentic AI（代理式AI）的崛起：**

   *   **描述：** AI正从被动工具转变为能自主感知、规划、行动和学习的“AI代理人”。它们能分解复杂任务，动态调整策略，实现更高级别的认知和决策能力。
   *   **主要公司：** Nvidia（黄仁勋提出概念）、Google、Microsoft、OpenAI。
   *   **潜在影响：** 极大提升工作流程自动化水平，重塑商业模式，并在自动驾驶、机器人等领域有广泛应用。

**2. 多模态大模型（Multimodal Large Models）的融合：**

   *   **描述：** 模型不再局限于单一数据类型（文本、图像、音频、视频），而是能够统一处理和生成多种模态信息，更接近人类的认知方式。
   *   **主要公司：** Google (Gemini)、Baidu (文心大模型5.0)、OpenAI。
   *   **潜在影响：** 提升AI的理解和生成能力，实现更自然的交互，推动科学研究（AI4S）和内容创作的进步。

**3. AI在健康领域的深度渗透与个性化：**

   *   **描述：** AI正从辅助诊断、药物研发，向个性化健康管理、慢病监测、亚健康干预等消费者侧应用深化。技术整合了多种数据源，提供定制化健康方案。
   *   **主要公司：** 百度（讯飞晓医）、华为云、迪安诊断、万达信息；医疗器械公司如波士顿科学。
   *   **潜在影响：** 提高医疗服务效率和准确性，降低成本，改善患者预后，并推动全民健康管理和预防医学的发展。
TechResearcher > **最新AI/ML趋势简报**

**1. Agentic AI（代理式AI）的崛起：**

   *   **描述：** AI正从被动工具转变为能自主感知、规划、行动和学习的“AI代理人”。它们能分解复杂任务，动态调整策略，实现更高级别的认知和决策能力。
   *   **主要公司：** Nvidia（黄仁勋提出概念）、Google、Microsoft、OpenAI。
  

# 示例 4：循环工作流 - 持续优化迭代循环

让我们构建一个包含两个代理的系统：
1. 写作代理 - 负责撰写短篇故事的初稿；
2. 评论代理 - 负责审阅并批评短篇故事，提出改进建议

In [116]:
# 该代理在开始时运行一次以创建初稿。
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story",  # Stores the first draft in the state.
)

print("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


In [117]:
# 该代理的唯一职责是提供反馈或批准信号。它没有任何工具。
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique",  # Stores the feedback in the state.
)

print("✅ critic_agent created.")

✅ critic_agent created.


现在，我们需要一种方法让循环能够根据评论者的反馈真正停止。LoopAgent本身并不能自动理解"APPROVED"意味着"停止"。我们需要一个代理来给它一个明确的信号来终止循环。
我们通过两部分来实现这一点：
1. 一个简单的Python函数，LoopAgent将其理解为"退出"信号。
2. 一个代理，当满足正确条件时可以调用该函数。
首先，你需要定义exit_loop函数：

In [118]:
# This is the function that the RefinerAgent will call to exit the loop.
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}


print("✅ exit_loop function created.")

✅ exit_loop function created.


为了让代理调用这个Python函数，我们将其封装在FunctionTool中。然后，我们创建一个拥有此工具的RefinerAgent。👉 注意其指令：这个代理是循环的"大脑"。它会读取来自CriticAgent的{critique}，并决定是（1）调用exit_loop工具还是（2）重写故事。

In [119]:
# This agent refines the story based on critique OR calls the exit_loop function.
refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    output_key="current_story",  # It overwrites the story with the new, refined version.
    tools=[
        FunctionTool(exit_loop)
    ],  # The tool is now correctly initialized with the function reference.
)

print("✅ refiner_agent created.")

✅ refiner_agent created.


然后我们将这些代理整合到一个循环代理下，该循环代理本身又嵌套在一个顺序代理中。
这一设计确保系统首先生成一个初始故事草稿，然后细化循环会运行到指定的最大迭代次数。

In [120]:
# The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,  # Prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop.
root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


In [121]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a short story about a lighthouse keeper who discovers a mysterious, glowing map"
)


 ### Created new session: debug_session_id

User > Write a short story about a lighthouse keeper who discovers a mysterious, glowing map
InitialWriterAgent > The salt spray kissed Elias’s weathered face as he climbed the winding stairs. For thirty years, the lantern room had been his world, a beacon against the unforgiving sea. Tonight, however, was different. Tucked beneath a loose floorboard, a rolled parchment pulsed with an eerie, internal luminescence. Unfurling it revealed not charts of known waters, but swirling, celestial patterns inked in a phosphorescent green. Islands, marked with symbols he’d never seen, floated on an astral sea. A thrumming began in his chest, a call from beyond the familiar fog. He traced a line with a trembling finger, a journey beckoning him to shores unknown, guided by a map that defied the very ocean he guarded.
InitialWriterAgent > The salt spray kissed Elias’s weathered face as he climbed the winding stairs. For thirty years, the lantern room had b

到这里，Day1 学习任务结束了，你现在已经具备了作为“智能体编排者”的基础能力，可以设计出由多个 AI 协作完成复杂任务的系统了！🎉
